In [15]:
# Visualization
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
!python create_pplmaps.py

Predicting on images in abraham_isaac
100%|███████████████████████████████████████████| 11/11 [00:06<00:00,  1.72it/s]


In [ ]:
# # Choose batch and model settings
# dirs = {
#     "imgs":"datasets/stories/imgs",
#     "maps":"datasets/stories/pplmaps",
#     "preds":"datasets/stories/predictions",
# }

# weights_path = './logs/peopleart_coco/efficientdet-d0_149_600.pth' # 150 epochs, last layer

# # for model
# threshold = 0.5
# iou_threshold = 0.5
# obj_list = ['person']

# compound_coef = 0
# use_cuda = True
# use_float16 = False

# # for pplmap
# downscale = 1 # along each img dimension
# map_shade = 64 # allows up to 4 boxes to overlap

# # Load model
# def load_model(weights_path, obj_list, compound_coef, use_cuda = True, use_float16 = False):
#     cudnn.fastest = True 
#     cudnn.benchmark = True

#     model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),
#                                  # TODO: replace this part with your project's anchor config
#                                  ratios=[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)],
#                                  scales=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

#     model.load_state_dict(torch.load(weights_path)) # TODO: just weight path?
#     model.requires_grad_(False)
#     model.eval()

#     if use_cuda:
#         model = model.cuda()
#     if use_float16:
#         model = model.half()
        
#     return model

# model = load_model(weights_path, obj_list, compound_coef, use_cuda, use_float16)

# # Process images 
# def format_for_prediction(img_path, compound_coef, use_cuda = True):
#     force_input_size = None  # set None to use default size
#     input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
#     input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size

#     ori_imgs, framed_imgs, framed_metas = preprocess(img_path, max_size=input_size)

#     if use_cuda:
#         x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
#     else:
#         x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

#     x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

#     return ori_imgs, framed_metas, x

# for story in ["adam_eve","last_supper", "abraham_isaac"]:
#     path = Path(os.path.join(dirs['imgs'], story))
#     batch_paths = [p.as_posix() for p in path.glob('*')]
    
#     for img_path in tqdm(batch_paths):
#         # Pre-process
#         ori_imgs, framed_metas, x = format_for_prediction(img_path, compound_coef, use_cuda)
    
#         # Predict
#         with torch.no_grad():
#             features, regression, classification, anchors = model(x)
#             regressBoxes = BBoxTransform()
#             clipBoxes = ClipBoxes()
#             out = postprocess(x,
#                               anchors, regression, classification,
#                               regressBoxes, clipBoxes,
#                               threshold, iou_threshold)

#         # Post-process
#         out = invert_affine(framed_metas, out)

#         for i in range(len(ori_imgs)):
#             img = ori_imgs[i].copy()
#             bbox_coords = out[i]['rois'].astype(np.int)

#             if not len(bbox_coords):
#                 print("No predicted boxes for img %s" % img_path)
#                 continue
            
#             # Single channel pplmap
#             mapw, maph = np.asarray(img.shape[:2]) // downscale
#             pplmap = np.zeros((map_w, map_h))
            
#             for j in range(len(bbox_coords)):
#                 (x1, y1, x2, y2) = bbox_coords[j]
#                 print(bbox_coords[j])
#                 # Update pplmap
#                 mapx1, mapy1, mapx2, mapy2 = np.array((x1, y1, x2, y2)) // downscale
#                 pplmap[mapy1:mapy2+1, mapx1:mapx2+1] += map_shade
                
#                 # Plot bbox predictions
#                 cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 0), 2)
#                 obj = obj_list[out[i]['class_ids'][j]]
#                 score = float(out[i]['scores'][j])

#                 cv2.putText(img, '{}, {:.3f}'.format(obj, score),
#                             (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
#                             (255, 255, 0), 1)
#                 plt.imshow(img)
            
#             # Save as files
#             map_path = os.path.join(dirs['maps'], story, os.path.basename(img_path))
#             cv2.imwrite(map_path, pplmap)
            
#             viz_path = os.path.join(dirs['preds'], story, os.path.basename(img_path))
#             plt.savefig(viz_path)
#             plt.close()
